<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Mathematics Basics

**With `NumPy`**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Case Study: LSM Algorithm

### Monte Carlo Simulation

In the Black-Scholes-Merton (1973) model based on the geometric Brownian motion, the future price $S_t$ of the risky stock is given in a discrete simulation context by

$$S_t = S_{t - \Delta t} \cdot \exp \left(\left(r - \frac{\sigma^2}{2} \right)\Delta t + \sigma \sqrt{\Delta t} z \right)$$

$S_0$ is the initial stock price, $r$ the risk-free short rate, $\sigma$ the volatility factor, $T>0$ a future point in time and $z$ a standard normally distributed rv. $\Delta t$ is the homogeneous time interval.

In [ ]:
!git clone https://github.com/tpq-classes/mathematics_basics.git
import sys
sys.path.append('mathematics_basics')


In [ ]:
import math
import numpy as np

In [ ]:
S0 = 36.
K = 40.
r = 0.06
T = 1.0
sigma = 0.2

### Simulating the Stock Price Process

In [ ]:
M = 50
I = 250000

In [ ]:
dt = T / M
dt

In [ ]:
df = math.exp(-r * dt)
df

In [ ]:
from numpy.random import default_rng
rng = default_rng(100)

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
rn = rng.standard_normal((M + 1, I))
rn.round(4)

In [ ]:
S = np.zeros_like(rn)
S[0] = S0
S

In [ ]:
for t in range(1, M + 1):
    S[t] = S[t - 1] * np.exp((r - sigma ** 2 / 2) * dt +
                           sigma * math.sqrt(dt) * rn[t])

In [ ]:
S.round(4)

In [ ]:
from pylab import mpl, plt
#äplt.style.available
plt.style.use('seaborn-v0_8')
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['savefig.dpi'] = 300

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(S[:, :10]);

In [ ]:
ST = S[-1]
plt.figure(figsize=(10, 6))
plt.hist(ST, bins=35, color='b', label='frequency');
plt.axvline(ST.mean(), color='r', label='mean')
plt.axvline(ST.mean() + ST.std(), ls='--', color='y', label='sd up')
plt.axvline(ST.mean() - ST.std(), ls='-.', color='y', label='sd down')
plt.legend(loc=0);

In [ ]:
S0 * math.exp(r * T)

In [ ]:
ST.mean()

### European Option Pricing

In [ ]:
h = np.maximum(K - ST, 0)  # put option
h

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(h, color='b', bins=35);

In [ ]:
math.exp(-r * T) * h.mean()

### American Option Pricing

For details see Hilpisch (2015): _Derivatives Analytics with Python_. Wiley Finance.

In [ ]:
h = np.maximum(K - S, 0)

In [ ]:
# Least-Squares Monte Carlo Valuation (LSM algorithm)
V = h[-1]
for t in range(M - 1, 0, -1):
    reg = np.polyfit(S[t], df * V, deg=5)
    C = np.polyval(reg, S[t])
    V = np.where(h[t] > C, h[t], df * V)

In [ ]:
df * V.mean()

### LSM as Function

In [ ]:
M, I = 50, 100000

In [ ]:
def time_first(M, I):
    dt = T / M
    df = math.exp(-r * dt)
    rng = default_rng(100)
    rn = rng.standard_normal((M + 1, I))
    rn = (rn - rn.mean()) / rn.std()
    S = np.zeros_like(rn)
    S[0] = S0
    for t in range(1, M + 1):
        S[t] = S[t - 1] * np.exp((r - sigma ** 2 / 2) * dt +
                sigma * math.sqrt(dt) * rn[t])
    h = np.maximum(K - S, 0)
    V = h[-1]
    for t in range(M - 1, 0, -1):
        reg = np.polyfit(S[t], df * V, deg=5)
        C = np.polyval(reg, S[t])
        V = np.where(h[t] > C, h[t], df * V)
    V0 = df * V.mean()
    return V0

In [ ]:
%time time_first(M, I)

In [ ]:
%timeit time_first(M, I)

### Full Vectorization

In [ ]:
def time_first(M, I):
    dt = T / M
    df = math.exp(-r * dt)
    rng = default_rng(100)
    rn = rng.standard_normal((M + 1, I))
    rn = (rn - rn.mean()) / rn.std()
    S = np.zeros_like(rn)
    S[0] = S0
    S[1:] = S0 * np.exp(((r - sigma ** 2 / 2) * dt +
                sigma * math.sqrt(dt) * rn[1:]).cumsum(axis=0))
    h = np.maximum(K - S, 0)
    V = h[-1]
    for t in range(M - 1, 0, -1):
        reg = np.polyfit(S[t], df * V, deg=5)
        C = np.polyval(reg, S[t])
        V = np.where(h[t] > C, h[t], df * V)
    V0 = df * V.mean()
    return V0

In [ ]:
%time time_first(M, I)

In [ ]:
%timeit time_first(M, I)

### Memory Layout

In [ ]:
def paths_first(I, M):
    dt = T / M
    df = math.exp(-r * dt)
    rng = default_rng(100)
    rn = rng.standard_normal((I, M + 1))
    rn = (rn - rn.mean()) / rn.std()
    S = np.zeros_like(rn)
    S[0] = S0
    S[:, 1:] = S0 * np.exp(((r - sigma ** 2 / 2) * dt +
                sigma * math.sqrt(dt) * rn[:, 1:]).cumsum(axis=1))
    h = np.maximum(K - S, 0)
    V = h[:, -1]
    for t in range(M - 1, 0, -1):
        reg = np.polyfit(S[:, t], df * V, deg=5)
        C = np.polyval(reg, S[:, t])
        V = np.where(h[:, t] > C, h[:, t], df * V)
    V0 = df * V.mean()
    return V0

In [ ]:
%time paths_first(I, M)

In [ ]:
%timeit paths_first(I, M)

### Separation

In [ ]:
def vectorized(M, I):
    dt = T / M
    rng = default_rng(100)
    rn = rng.standard_normal((M + 1, I))
    S = np.zeros_like(rn)
    S[0] = S0
    S[1:] = S0 * np.exp(((r - sigma ** 2 / 2) * dt +
                sigma * math.sqrt(dt) * rn[1:]).cumsum(axis=0))
    return S

In [ ]:
def lsm_algo(M, I, func):
    dt = T / M
    df = math.exp(-r * dt)
    S = func(M, I)
    h = np.maximum(K - S, 0)
    V = h[-1]
    for t in range(M - 1, 0, -1):
        reg = np.polyfit(S[t], df * V, deg=5)
        C = np.polyval(reg, S[t])
        V = np.where(h[t] > C, h[t], df * V)
    V0 = df * V.mean()
    return V0

In [ ]:
%time lsm_algo(M, I, vectorized)

In [ ]:
%timeit lsm_algo(M, I, vectorized)

### Dynamic Compilation

In [ ]:
import numba
import random

In [ ]:
def nested_loops(M, I):
    dt = T / M
    S = np.zeros((M + 1, I))
    S[0] = S0
    for t in range(1, M + 1):
        for i in range(0, I):
            S[t, i] = S[t - 1, i] * math.exp((r - sigma ** 2 / 2) * dt +
                sigma * math.sqrt(dt) * random.gauss(0, 1))
    return S

In [ ]:
%time lsm_algo(M, I, nested_loops)

In [ ]:
nested_loops_nb = numba.jit(nested_loops)

In [ ]:
%time lsm_algo(M, I, nested_loops_nb)

In [ ]:
%time lsm_algo(M, I, nested_loops_nb)

In [ ]:
%timeit lsm_algo(M, I, nested_loops_nb)

### Static Compilation

In [ ]:
%load_ext Cython

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
from numpy.random import default_rng
rng = default_rng()
from libc.math cimport exp, sqrt
cdef float S0 = 36.
cdef float T = 1.0
cdef float r = 0.06
cdef float sigma = 0.2
@cython.boundscheck(False)
@cython.wraparound(False)
def nested_loops_cy(int M, int I):
    cdef int t, i
    cdef float dt = T / M
    cdef double[:, :] S = np.zeros((M + 1, I))
    cdef double[:, :] rn = rng.standard_normal((M + 1, I))
    S[0] = S0
    for t in range(1, M + 1):
        for i in range(I):
            S[t, i] = S[t - 1, i] * exp((r - sigma ** 2 / 2) * dt +
                                        sigma * sqrt(dt) * rn[t, i])
    return np.array(S)

In [ ]:
%time lsm_algo(M, I, nested_loops_cy)

In [ ]:
%timeit lsm_algo(M, I, nested_loops_cy)

### Comparison 

In [ ]:
M, I = 50, 100000

In [ ]:
%time S = nested_loops(M, I)

In [ ]:
%time S = vectorized(M, I)

In [ ]:
%time S = nested_loops_nb(M, I)

In [ ]:
%time S = nested_loops_cy(M, I)

In [ ]:
M, I = 50, 200000

In [ ]:
%timeit S = vectorized(M, I)

In [ ]:
%timeit S = nested_loops_nb(M, I)

In [ ]:
%timeit S = nested_loops_cy(M, I)

<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>